<center> <h1> Customer Segmentation in Banking </h1> </center>
<p style="margin-bottom:1cm;"></p>

# Dependencies

In [1]:
%matplotlib inline
%load_ext lab_black

In [2]:
!pip install imblearn --ignore-installed

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\juana_0mrk5mk\\anaconda3\\envs\\MLG\\Lib\\site-packages\\numpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
from configparser import ConfigParser
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support,
    precision_recall_curve,
    f1_score,
    classification_report,
)
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

from imblearn.under_sampling import RandomUnderSampler

In [5]:
# to use the seaborn style in the matplotlib plots
plt.style.use("seaborn")

## Data understanding

Load the data and have a look at the basic properties.

In [6]:
parser = ConfigParser()
_ = parser.read("notebook.cfg")

path = parser.get("my_api", "path")

In [8]:
data_path = path + "/Data/banking.csv"

In [9]:
df_raw = pd.read_csv(data_path)
df = df_raw.copy()
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [ ]:
# Check the type of values of each column
df.dtypes

In [ ]:
# Check the shape of the DataFrame
df.shape

In [ ]:
# Check if the df has NaN values
df.isna().sum()

In [ ]:
# Some information about each feature
df.describe()

In [ ]:
# This creats histograms for all numerical features
df.hist(figsize=(15, 10))
None

In [ ]:
# Let's check the number of categories per categorical feature
df.select_dtypes(include=["object"]).apply(lambda col: len(col.unique()))

In [ ]:
# Take a closer look at features with many categories
print(f"Job categories:\n{df['job'].unique()}\n")
print(f"Month categories:\n{df['month'].unique()}\n")
print(f"Education categories:\n{df['education'].unique()}")

In general, the categories look pretty clean. For the education categories we can combine the different levels of basic education to reduce the number of categories.

## Data preprocessing

### Feature extraction 

Don't use the feature 'duration' for building a model since the description of this feature states: 

Last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

In [ ]:
# check the name of the columns
df.columns

In [ ]:
# get features and corresponding outcomes
df_clean = df.drop(columns=["duration"])
df_clean

### Train-Test Split

We separate the dataset into train and test as follows in order to perform sampling later.   
Taking sample from the whole dataset (with 'y' included) is easier than taking it separately from X_train and y_train.  

In [16]:
df_train, df_test = train_test_split(
    df_clean, test_size=0.2, random_state=5, stratify=df_clean["y"]
)

### Split numerical and categorical features
For data preprocessing we have to separate numerical and categorical features. Different preprocessing steps are required for both type of features.

In [ ]:
categorical_features = df.iloc[:, list(np.where(X.dtypes == "object")[0])].columns
numerical_features = df.iloc[:, list(np.where(X.dtypes == "int")[0])].columns
categorical_features, numerical_features